<a href="https://colab.research.google.com/github/Jebina-ai/AI-EMOTION-DETECTION/blob/master/respbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
from tensorflow.keras.utils import to_categorical
import string
import re
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#Importing dataset
from google.colab import files
data = files.upload()

Saving test.txt to test.txt
Saving train.txt to train.txt
Saving val.txt to val.txt


In [4]:
data=pd.read_csv("train.txt",header=None,sep=';',names=['Text','Sentiments'])
data

,Text,Sentiments
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [5]:
data=data.dropna()
data.shape

(16000, 2)

In [6]:
data['Sentiments'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [7]:
data['Sentiments']=data.Sentiments.replace({'sadness':0,'joy':1,'love':2,'anger':3,'surprise':4,'fear':5})

In [8]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [9]:
X=data['Text']

In [10]:
tokenizer=Tokenizer(15212,lower=True,oov_token='UNK')

In [11]:
tokenizer.fit_on_texts(X)

In [12]:
X_train=tokenizer.texts_to_sequences(X)

In [13]:
X_train_paded=pad_sequences(X_train,maxlen=80,padding='pre')

In [14]:
Y_train=data['Sentiments'].values

In [15]:
Y_train_data=to_categorical(Y_train)

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,LSTM,Bidirectional
from tensorflow.keras.layers import Embedding

In [17]:
model=Sequential()
#(vocab_size,feature_size,input_shape(maxleng))
model.add(Embedding(15212,64,input_length=80))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(80,return_sequences=True)))
model.add(Bidirectional(LSTM(160)))

model.add(Dense(6,activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 64)            973568    
                                                                 
 dropout (Dropout)           (None, 80, 64)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 80, 160)          92800     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 320)              410880    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 1926      
                                                                 
Total params: 1,479,174
Trainable params: 1,479,174
Non-

In [19]:
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [20]:
model.fit(X_train_paded,Y_train_data,epochs=10)

Epoch 1/10
500/500 [==============================] - 224s 435ms/step - loss: 1.2517 - accuracy: 0.5163
Epoch 2/10
500/500 [==============================] - 215s 431ms/step - loss: 0.5136 - accuracy: 0.8186
Epoch 3/10
500/500 [==============================] - 216s 433ms/step - loss: 0.2740 - accuracy: 0.9072
Epoch 4/10
500/500 [==============================] - 217s 433ms/step - loss: 0.1981 - accuracy: 0.9280
Epoch 5/10
500/500 [==============================] - 215s 430ms/step - loss: 0.1643 - accuracy: 0.9380
Epoch 6/10
500/500 [==============================] - 216s 432ms/step - loss: 0.1305 - accuracy: 0.9513
Epoch 7/10
500/500 [==============================] - 216s 431ms/step - loss: 0.1171 - accuracy: 0.9561
Epoch 8/10
500/500 [==============================] - 216s 432ms/step - loss: 0.0974 - accuracy: 0.9635
Epoch 9/10
500/500 [==============================] - 216s 433ms/step - loss: 0.0876 - accuracy: 0.9661
Epoch 10/10
500/500 [==============================] - 216s 433m

In [21]:
def get_key(value):
  dict={'sadness':0,'joy':1,'love':2,'anger':3,'surprise':4,'fear':5}
  for i, j in dict.items():
    if(j==value):
      return i

In [22]:
def predict(sentence):
  pred_data=[]
  pred_data.append(sentence)
  # print(pred_data)
  sentence_seq=tokenizer.texts_to_sequences(pred_data)
  # print(sentence_seq)
  padded_pred=pad_sequences(sentence_seq,maxlen=80,padding='pre')
  predicted=model.predict(padded_pred)
  results=np.argmax(predicted,axis=1)
  result=get_key(results)
  return result

In [25]:
from IPython.display import  clear_output
input_sentimentals=print("Hi,What is your name?")
user=input()
input_sentimental=input("How are you feeling today?")
result=predict(input_sentimental)
if result=="sadness":
  print("Tell me about it "+user+'?')
  response=input()
  print("Hey "+user +"I am here for you. Everyone has a hard time in their life at some point we need to have hope")
  assistance=input("Do you need any assitance?")
  if assistance=='n' or assistance=='no' or assistance=='No' or assistance=='N':
    print("Bye,Bye " + user + " Have a nice Day")
  elif assistance=='y' or assistance=='yes' or assistance=='Y' or assistance=='Yes':
    print("How can i help you?")
    print("\n")
    print("1.Do you want to talk more?")
    print("2.Do you want to listen music?")
    response=input()
    if response=="1":
      print(user+"Yeah Im listening")
    elif response=="2":
   
      print(" ")
elif result=="anger":
  print("Dear" +" "+user+" "+"Anger dosen't solve anything, But it destroys everything" )
  print("To avoid your anger please do excerise...."+" ")
  bot=input("Do you want to do Excerise?")
  if bot=='n' or bot=='no' or bot=='No' or bot=='N':
    print("Bye,Bye " + user + " Have a great day")
  elif bot=='y' or bot=='yes' or bot=='Y' or bot=='Yes':
    print("Please do the Excerise Daily and control your anger"+" "+'https://positivepsychology.com/anger-management-techniques/')
    print("Bye"+" "+user+" "+"Have a nice day.")
elif result=="fear":
  print("Hey"+user+"You dont have to be afraid")
else: 
  print("Oh " + user + " Be happy. Bye!")

Hi,What is your name?
helen
How are you feeling today?angry
Dear helen Anger dosen't solve anything, But it destroys everything
To avoid your anger please do excerise.... 
Do you want to do Excerise?y
Please do the Excerise Daily and control your anger https://positivepsychology.com/anger-management-techniques/
Bye helen Have a nice day.
